#Support Vector Machines - Spam Classification

This Matlab code has been completed as part of [Machine Learning](https://www.coursera.org/learn/machine-learning)
course in Coursera given by Prof. Andrew Ng

------------
This note demonstrates the use of SVM for spam classification.

Many email services today provide spam filters that are able to classify emails into spam and non-spam email with high accuracy. This note demonstrates the use of SVMs to build a spam filter. We will be training a classifier to classify whether a given email, x, is spam (y = 1) or non-spam (y = 0). To do so, we will need to convert each email into a feature vector $x \in R^n$ for training a SVM classifier. 

This notebook includes:

- <a href='#part1'>Part 1: Email Preprocessing</a>
- <a href='#part2'>Part 2: Part 2: Train Linear SVM for Spam Classificationn</a>

This code requires the following functions
* <a href='#funcs_08_SupportVectorMachine_readFile.m'>funcs_08_SupportVectorMachine_readFile.m</a> 
* <a href='#funcs_08_SupportVectorMachine_processEmail.m'>funcs_08_SupportVectorMachine_processEmail.m</a>
* <a href='#funcs_08_SupportVectorMachine_porterStemmer.m'>funcs_08_SupportVectorMachine_porterStemmer.m</a>
* <a href='#funcs_08_SupportVectorMachine_svmTrain.m'>funcs_08_SupportVectorMachine_svmTrain.m</a>
* <a href='#funcs_08_SupportVectorMachine_svmPredict.m'>funcs_08_SupportVectorMachine_svmPredict.m</a>

In [11]:
clear all; close all; clc; warning off;
addpath('./data/') % add path to directory containing data files
addpath('./funcs/') % add path to directory containing subfunction files

## Part 1: Email Preprocessing <a id='part1'></a>

Before starting on a machine learning task, it is usually insightful to take a look at examples from the dataset.

In [12]:
filename = 'data_08_SuportVectorMachine_part2_emailSample1.txt';
file_contents = funcs_08_SupportVectorMachine_readFile(filename);
fprintf('%s ', file_contents);

> Anyone knows how much it costs to host a web portal ?
>
Well, it depends on how many visitors you're expecting.
This can be anywhere from less than 10 bucks a month to a couple of $100. 
You should checkout http://www.rackspace.com/ or perhaps Amazon EC2 
if youre running something big..

To unsubscribe yourself from this mailing list, send an email to:
groupname-unsubscribe@egroups.com

The sample email contains a URL, an email address (at the end), numbers, and dollar amounts. While many emails would contain similar types of entities (e.g., numbers, other URLs, or other email addresses), the specific entities (e.g., the specific URL or specific dollar amount) will be different in almost every
email. Therefore, one method often employed in processing emails is to "normalize" these values, so that all URLs are treated the same, all numbers are treated the same, etc. For example, we could replace each URL in the email with the unique string "httpaddr" to indicate that a URL was present. This has the effect of letting the spam classifier make a classification decision based on whether any URL was present, rather than whether a specific URL
was present. This typically improves the performance of a spam classifier, since spammers often randomize the URLs, and thus the odds of seeing any particular URL again in a new piece of spam is very small.

In processEmail.m, we have implemented the following email preprocessing and normalization steps:
* **Lower-casing:** The entire email is converted into lower case, so that captialization is ignored (e.g., IndIcaTE is treated the same as Indicate).
* **Stripping HTML:** All HTML tags are removed from the emails. Many emails often come with HTML formatting; we remove all the HTML tags, so that only the content remains.
* **Normalizing URLs:** All URLs are replaced with the text *"httpaddr"*.
* **Normalizing Email Addresses:** with the text *"emailaddr"*.
* **Normalizing Numbers:** All email addresses are replaced All numbers are replaced with the text *"number"*.
* **Normalizing Dollars:** All dollar signs ($) are replaced with the text *"dollar"*.
* **Word Stemming:** Words are reduced to their stemmed form. For example, *"discount"*, *"discounts"*, *"discounted"* and *"discounting"* are all replaced with *"discount"*. Sometimes, the Stemmer actually strips off additional characters from the end, so *"include"*, *"includes"*, *"included"*, and *"including"* are all replaced with *"includ"*. 
* **Removal of non-words:** Non-words and punctuation have been removed. All white spaces (tabs, newlines, spaces) have all been trimmed to a single space character.

The result of these preprocessing steps is shown in Figure 9. While pre-processing has left word fragments and non-words, this form turns out to be much easier to work with for performing feature extraction.

#### Vocabulary List:
After preprocessing the emails, we have a list of words for each email. The next step is to choose which words we would like to use in our classifier and which we would want to leave out. For this test, we have chosen only the most frequently occuring words as our set of words considered (the vocabulary list). Since words that occur
rarely in the training set are only in a few emails, they might cause the model to overfit our training set. The complete vocabulary list is in the file *vocab.txt*. This vocabulary list was selected by choosing all words which occur at least a 100 times in the spam corpus, resulting in a list of 1899 words. In practice, a vocabulary list with about 10,000 to 50,000 words is often used. 

Given the vocabulary list, we can now map each word in the preprocessed emails (e.g., Figure 9) into a list of word indices that contains the index of the word in the vocabulary list.

In [13]:
% Load Vocabulary
fid = fopen('data_08_SuportVectorMachine_part2_vocab.txt');

% Store all dictionary words in cell array vocab{}
n = 1899;  % Total number of words in the dictionary

% For ease of implementation, we use a struct to map the strings => integers
% In practice, you'll want to use some form of hashmap
vocabList = cell(n, 1);
for i = 1:n
    % Word Index (can ignore since it will be = i)
    fscanf(fid, '%d', 1);
    % Actual Word
    vocabList{i} = fscanf(fid, '%s', 1);
end
fclose(fid);

In [14]:
% postprocess email
word_indices  = funcs_08_SupportVectorMachine_processEmail(file_contents,vocabList);

% Print Stats
fprintf('Word Indices: \n');
fprintf(' %d', word_indices);
fprintf('\n\n');

==== Processed Email ====

anyon know how much it cost to host a web portal well it depend on how mani 
visitor you re expect thi can be anywher from less than number buck a month 
to a coupl of dollarnumb you should checkout httpaddr or perhap amazon ecnumb 
if your run someth big to unsubscrib yourself from thi mail list send an 
email to emailaddr 

Word Indices: 
 86 916 794 1077 883 370 1699 790 1822 1831 883 431 1171 794 1002 1893 1364 592 1676 238 162 89 688 945 1663 1120 1062 1699 375 1162 479 1893 1510 799 1182 1237 810 1895 1440 1547 181 1699 1758 1896 688 1676 992 961 1477 71 530 1699 531

### Extracting Features from Emails

Now, you will convert each email into a vector of features in $R^n$. We will be using $n = \# words$ in vocabulary
list. Specifically, the feature $x_i \in \{0, 1\}$ for an email corresponds to whether the $i^{th}$ word in the dictionary occurs in the email. That is, $x_i = 1$ if the $i^{th}$ word is in the email and $x_i = 0$ if the $i^{th}$ word is not present in the email. For a typical email, this feature would look like

$$x=[0 \: 0 \: 1 \: ... \: 1 \: 0 \: ... \: 0]^T \in R^n$$

In [15]:
features = zeros(n, 1);
features(word_indices)=1;
% Print Stats
fprintf('Length of feature vector: %d\n', length(features));
fprintf('Number of non-zero entries: %d\n', sum(features > 0));

Length of feature vector: 1899
Number of non-zero entries: 45

## Part 2: Train Linear SVM for Spam Classification   <a id='part2'></a>

After you have completed the feature extraction functions, the next step is to load a preprocessed training dataset that will be used to train a SVM classifier. The file *data_08_SuportVectorMachine_part2_spamTrain.mat* contains 4000 training examples of spam and non-spam email, while *data_08_SuportVectorMachine_part2_spamTest* contains 1000 test examples. Each original email was processed and converted into a vector $x^{(i)} \in R^{1899}$.

In [16]:
load('./data/data_08_SuportVectorMachine_part2_spamTrain.mat');

C = 0.1;
svm_model = funcs_08_SupportVectorMachine_svmTrain(X, y, C, @linearKernel);

pred_train = funcs_08_SupportVectorMachine_svmPredict(svm_model, X);
fprintf('Training Accuracy: %f\n', mean(double(pred_train == y)) * 100);

Training Accuracy: 99.800000

In [17]:
load('./data/data_08_SuportVectorMachine_part2_spamTest.mat');
pred_test = funcs_08_SupportVectorMachine_svmPredict(svm_model, Xtest);
fprintf('Testing Accuracy: %f\n', mean(double(pred_test == ytest)) * 100);

Testing Accuracy: 98.700000

#### Top Predictors for Spam
To better understand how the spam classifier works, we can inspect the parameters to see which words the classifier thinks are the most predictive of spam. We finds the parameters with the largest positive values in the classifier. For example, if an email contains words such as "guarante", "remove", "dollar", and "price" (the top predictors), it is likely to be classified as spam.

In [18]:
[weight, idx] = sort(svm_model.w, 'descend');

fprintf('\nTop predictors of spam: \n');
for i = 1:15
    fprintf(' %-15s (%f) \n', vocabList{idx(i)}, weight(i));
end

Top predictors of spam: 
 our             (0.500005) 
 click           (0.466328) 
 remov           (0.424416) 
 guarante        (0.386305) 
 visit           (0.368418) 
 basenumb        (0.347584) 
 dollar          (0.331071) 
 will            (0.269945) 
 price           (0.263403) 
 pleas           (0.260972) 
 lo              (0.260926) 
 nbsp            (0.257982) 
 most            (0.252809) 
 ga              (0.248242) 
 hour            (0.241250)

### Try sample emails

In [19]:
filename = './data/data_08_SuportVectorMachine_part2_spamSample1.txt';
file_contents = funcs_08_SupportVectorMachine_readFile(filename);
word_indices  = funcs_08_SupportVectorMachine_processEmail(file_contents,vocabList);
x = zeros(n, 1);
x(word_indices)=1;

p = funcs_08_SupportVectorMachine_svmPredict(svm_model, x);

fprintf('\nProcessed %s\n\nSpam Classification: %d\n', filename, p);
fprintf('(1 indicates spam, 0 indicates not spam)\n\n');

==== Processed Email ====

do you want to make dollarnumb or more per week if you ar a motiv and qualifi 
individu i will person demonstr to you a system that will make you dollarnumb 
number per week or more thi is not mlm call our number hour pre record number 
to get the detail number number number i need peopl who want to make seriou 
monei make the call and get the fact invest number minut in yourself now 
number number number look forward to your call and i will introduc you to 
peopl like yourself who ar current make dollarnumb number plu per week number 
number number numberljgvnumb numberleannumberlrmsnumb 
numberwxhonumberqiytnumb numberrjuvnumberhqcfnumb numbereidbnumberdmtvlnumb 


Processed ./data/data_08_SuportVectorMachine_part2_spamSample1.txt

Spam Classification: 1
(1 indicates spam, 0 indicates not spam)

In [20]:
filename = './data/data_08_SuportVectorMachine_part2_spamSample2.txt';
file_contents = funcs_08_SupportVectorMachine_readFile(filename);
word_indices  = funcs_08_SupportVectorMachine_processEmail(file_contents,vocabList);
x = zeros(n, 1);
x(word_indices)=1;

p = funcs_08_SupportVectorMachine_svmPredict(svm_model, x);

fprintf('\nProcessed %s\n\nSpam Classification: %d\n', filename, p);
fprintf('(1 indicates spam, 0 indicates not spam)\n\n');

==== Processed Email ====

best bui viagra gener onlin viagra numbermg x number pill dollarnumb free 
pill reorder discount top sell number qualiti satisfact guarante we accept 
visa master e check payment number satisfi custom httpaddr 


Processed ./data/data_08_SuportVectorMachine_part2_spamSample2.txt

Spam Classification: 1
(1 indicates spam, 0 indicates not spam)

### Todo list:

* Build a dataset of training and testing set using the original emails from the SpamAssassin Public Corpus: Download the original files from the public corpus and extract them. After extracting, processing and extracting features from each email to extract a feature vector from each email. This will allow you to build a
dataset X, y of examples. You should then randomly divide up the dataset into a training set, a cross validation set and a test set. While you are building your own dataset, we also encourage you to try building your own vocabulary list (by selecting the high frequency words 

The original emails will have email headers that you might wish to leave out. We have included code in processEmail that will help you remove these headers and adding any additional features that you think might be useful. 

* Finally, we also suggest trying to use highly optimized SVM toolboxes such as LIBSVM